In [ ]:
import torch
from torch import nn
from torch.nn.functional import cross_entropy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch_dwn as dwn

In [ ]:
device = 'cuda:0'

### Train and evaluate functions

In [ ]:
def evaluate(model, x_test, y_test):
    model.eval()
    with torch.no_grad():
        pred = (model(x_test.cuda(device)).cpu()).argmax(dim=1).numpy()
        acc = (pred == y_test.numpy()).sum() / y_test.shape[0]
    return acc

def train_and_evaluate(model, optimizer, scheduler, x_train, y_train, x_test, y_test, epochs, batch_size):
    n_samples = x_train.shape[0]
    
    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(n_samples)
        correct_train = 0
        total_train = 0
        
        for i in range(0, n_samples, batch_size):
            optimizer.zero_grad()
            
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = x_train[indices].cuda(device), y_train[indices].cuda(device)
            
            outputs = model(batch_x)
            loss = cross_entropy(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            pred_train = outputs.argmax(dim=1)
            correct_train += (pred_train == batch_y).sum().item()
            total_train += batch_y.size(0)
        
        train_acc = correct_train / total_train
        
        scheduler.step()
        
        test_acc = evaluate(model, x_test, y_test)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')

# MNIST

### Load MNIST

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.flatten(x))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=len(train_dataset), shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)

x_train, y_train = next(iter(train_loader))
x_test, y_test = next(iter(test_loader))

### Binarize data with Distributive Thermometer

In [ ]:
thermometer = dwn.DistributiveThermometer(3).fit(x_train)
x_train = thermometer.binarize(x_train).flatten(start_dim=1)
x_test = thermometer.binarize(x_test).flatten(start_dim=1)

### Model

In [ ]:
model = nn.Sequential(
    dwn.LUTLayer(x_train.size(1), 2000, n=6, mapping='learnable'),
    dwn.LUTLayer(2000, 1000, n=6),
    dwn.GroupSum(k=10, tau=1/0.3)
).cuda(device)

### Optimizer and Scheduler

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=14)

### Train and evaluate model

In [4]:
train_and_evaluate(model, optimizer, scheduler, x_train, y_train, x_test, y_test, epochs=30, batch_size=32)

Epoch 1/30, Train Loss: 0.0783, Train Accuracy: 0.9187, Test Accuracy: 0.9606
Epoch 2/30, Train Loss: 0.0890, Train Accuracy: 0.9674, Test Accuracy: 0.9695
Epoch 3/30, Train Loss: 0.0225, Train Accuracy: 0.9773, Test Accuracy: 0.9728
Epoch 4/30, Train Loss: 0.0484, Train Accuracy: 0.9847, Test Accuracy: 0.9748
Epoch 5/30, Train Loss: 0.0561, Train Accuracy: 0.9891, Test Accuracy: 0.9781
Epoch 6/30, Train Loss: 0.0656, Train Accuracy: 0.9921, Test Accuracy: 0.9779
Epoch 7/30, Train Loss: 0.0521, Train Accuracy: 0.9943, Test Accuracy: 0.9791
Epoch 8/30, Train Loss: 0.0021, Train Accuracy: 0.9958, Test Accuracy: 0.9795
Epoch 9/30, Train Loss: 0.0053, Train Accuracy: 0.9971, Test Accuracy: 0.9801
Epoch 10/30, Train Loss: 0.0035, Train Accuracy: 0.9976, Test Accuracy: 0.9807
Epoch 11/30, Train Loss: 0.0154, Train Accuracy: 0.9979, Test Accuracy: 0.9800
Epoch 12/30, Train Loss: 0.0152, Train Accuracy: 0.9991, Test Accuracy: 0.9813
Epoch 13/30, Train Loss: 0.0121, Train Accuracy: 0.9991, Test

# JSC

In [ ]:
import openml
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Load Data

In [ ]:
dataset = openml.datasets.get_dataset(42468)
df_features, df_labels, _, attribute_names = dataset.get_data(dataset_format='dataframe', target=dataset.default_target_attribute)
features = df_features.values.astype(np.float32)
label_names = list(df_labels.unique())
labels = np.array(df_labels.map(lambda x : label_names.index(x)).values)
num_output = labels.max() + 1

### Train test split following LogicNets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, train_size=0.8, random_state=42)

### Binarize data with Distributive Thermometer

In [ ]:
thermometer = dwn.DistributiveThermometer(200).fit(x_train)
x_train = thermometer.binarize(x_train).flatten(start_dim=1)
x_test = thermometer.binarize(x_test).flatten(start_dim=1)

In [ ]:
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

### JSC Small

In [ ]:
model = nn.Sequential(
    dwn.LUTLayer(x_train.size(1), 50, n=6, mapping='learnable'),
    dwn.GroupSum(k=num_output, tau=1/0.3)
).cuda(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=14)

In [5]:
train_and_evaluate(model, optimizer, scheduler, x_train, y_train, x_test, y_test, epochs=30, batch_size=100)

Epoch 1/30, Train Loss: 0.8877, Train Accuracy: 0.7342, Test Accuracy: 0.7344
Epoch 2/30, Train Loss: 0.8222, Train Accuracy: 0.7386, Test Accuracy: 0.7342
Epoch 3/30, Train Loss: 0.8205, Train Accuracy: 0.7391, Test Accuracy: 0.7385
Epoch 4/30, Train Loss: 0.9186, Train Accuracy: 0.7393, Test Accuracy: 0.7382
Epoch 5/30, Train Loss: 0.9154, Train Accuracy: 0.7394, Test Accuracy: 0.7381
Epoch 6/30, Train Loss: 0.7503, Train Accuracy: 0.7396, Test Accuracy: 0.7388
Epoch 7/30, Train Loss: 0.7493, Train Accuracy: 0.7394, Test Accuracy: 0.7385
Epoch 8/30, Train Loss: 0.8439, Train Accuracy: 0.7396, Test Accuracy: 0.7372
Epoch 9/30, Train Loss: 0.6671, Train Accuracy: 0.7399, Test Accuracy: 0.7378
Epoch 10/30, Train Loss: 0.8610, Train Accuracy: 0.7398, Test Accuracy: 0.7375
Epoch 11/30, Train Loss: 0.7591, Train Accuracy: 0.7399, Test Accuracy: 0.7377
Epoch 12/30, Train Loss: 0.7076, Train Accuracy: 0.7399, Test Accuracy: 0.7371
Epoch 13/30, Train Loss: 0.7291, Train Accuracy: 0.7402, Test

### JSC Medium

In [ ]:
model = nn.Sequential(
    dwn.LUTLayer(x_train.size(1), 1000, n=6, mapping='learnable'),
    dwn.GroupSum(k=num_output, tau=1/0.05)
).cuda(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=14)

In [6]:
train_and_evaluate(model, optimizer, scheduler, x_train, y_train, x_test, y_test, epochs=30, batch_size=100)

Epoch 1/30, Train Loss: 0.8633, Train Accuracy: 0.7487, Test Accuracy: 0.7527
Epoch 2/30, Train Loss: 0.6423, Train Accuracy: 0.7552, Test Accuracy: 0.7543
Epoch 3/30, Train Loss: 0.6220, Train Accuracy: 0.7568, Test Accuracy: 0.7559
Epoch 4/30, Train Loss: 0.6660, Train Accuracy: 0.7585, Test Accuracy: 0.7559
Epoch 5/30, Train Loss: 0.5762, Train Accuracy: 0.7590, Test Accuracy: 0.7554
Epoch 6/30, Train Loss: 0.6594, Train Accuracy: 0.7592, Test Accuracy: 0.7554
Epoch 7/30, Train Loss: 0.6513, Train Accuracy: 0.7593, Test Accuracy: 0.7574
Epoch 8/30, Train Loss: 0.6353, Train Accuracy: 0.7597, Test Accuracy: 0.7575
Epoch 9/30, Train Loss: 0.6579, Train Accuracy: 0.7602, Test Accuracy: 0.7583
Epoch 10/30, Train Loss: 0.5930, Train Accuracy: 0.7605, Test Accuracy: 0.7570
Epoch 11/30, Train Loss: 0.7807, Train Accuracy: 0.7606, Test Accuracy: 0.7573
Epoch 12/30, Train Loss: 0.5843, Train Accuracy: 0.7604, Test Accuracy: 0.7577
Epoch 13/30, Train Loss: 0.5980, Train Accuracy: 0.7606, Test